# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: UNINASSAU



In [1]:
!pip install scikit-learn --upgrade

Requirement already up-to-date: scikit-learn in /usr/local/lib/python3.6/dist-packages (0.23.2)


In [2]:
# Importando as diversas bibliotecas que serão utilizadas:

# Pacote para trabalhar com JSON
import json

# Pacote para realizar requisições HTTP
import requests

# Pacote para exploração e análise de dados
import pandas as pd

# Pacote com métodos numéricos e representações matriciais
import numpy as np

# Pacotes do scikit-learn para pré-processamento de dados
# "SimpleImputer" é uma transformação para preencher valores faltantes em conjuntos de dados
from sklearn.impute import SimpleImputer

# Pacotes do scikit-learn para treinamento de modelos e construção de pipelines
# Método para separação de conjunto de dados em amostras de treino e teste
from sklearn.model_selection import train_test_split
# Método para criação de modelos 
from xgboost.sklearn import XGBClassifier

# Módulo para avaliação da qualidade de precisão do modelo
from sklearn import metrics


# Classe para a criação de uma pipeline de machine-learning
from sklearn.pipeline import Pipeline


In [3]:
# instalando o pandas profiling
#!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [4]:
# import o ProfileReport
#from pandas_profiling import ProfileReport

In [5]:
data = pd.read_csv("/content/drive/My Drive/dataset_desafio_2.csv")

In [6]:
# executando o profile
#profile = ProfileReport(data, title='Relatório - Pandas Profiling', html={'style':{'full_width':True}})

In [7]:
# salvando o relatório no disco
#profile.to_file(output_file="Relatorio01.html")

In [8]:
data.head()

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


In [9]:
# Percentual de valores faltantes do meu conjunto de dados
faltantes_pecentual = (data.isnull().sum() / len(data["NOME"]))*100
print(faltantes_pecentual)

MATRICULA          0.00
NOME               0.00
REPROVACOES_DE     0.00
REPROVACOES_EM     0.00
REPROVACOES_MF     0.00
REPROVACOES_GO     0.00
NOTA_DE            0.00
NOTA_EM            0.00
NOTA_MF            0.00
NOTA_GO           18.58
INGLES            18.14
H_AULA_PRES        0.00
TAREFAS_ONLINE     0.00
FALTAS             0.00
PERFIL             0.00
dtype: float64


In [10]:
from sklearn.base import BaseEstimator, TransformerMixin

# All sklearn Transforms must have the `transform` and `fit` methods
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        # Retornamos um novo dataframe sem as colunas indesejadas
        return data.drop(labels=self.columns, axis='columns')

In [11]:
# Instanciando uma transformação DropColumns
# Essa transformação recebe como parâmetro uma lista com os nomes das colunas indesejadas
rm_columns = DropColumns(
    columns=["NOME","MATRICULA", "INGLES"]  
)

print(rm_columns)

DropColumns(columns=['NOME', 'MATRICULA', 'INGLES'])


In [12]:
# Visualizando as colunas do dataset original
print("Colunas do dataset original: \n")
print(data.columns)

Colunas do dataset original: 

Index(['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM',
       'REPROVACOES_MF', 'REPROVACOES_GO', 'NOTA_DE', 'NOTA_EM', 'NOTA_MF',
       'NOTA_GO', 'INGLES', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS',
       'PERFIL'],
      dtype='object')


In [13]:
# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=data)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_data_2 = pd.DataFrame.from_records(
    data=rm_columns.transform(
        X=data
    ),
)

In [14]:
# Visualizando as colunas do dataset transformado
print("Colunas do dataset após a transformação ``DropColumns``: \n")
print(df_data_2.columns)

Colunas do dataset após a transformação ``DropColumns``: 

Index(['REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO',
       'NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'H_AULA_PRES',
       'TAREFAS_ONLINE', 'FALTAS', 'PERFIL'],
      dtype='object')


In [15]:
# Criação de um objeto ``SimpleImputer``
si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=0,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

In [16]:
df_data_2.isnull().sum(axis = 0) #Quantidade de valores faltantes em cada coluna.

REPROVACOES_DE       0
REPROVACOES_EM       0
REPROVACOES_MF       0
REPROVACOES_GO       0
NOTA_DE              0
NOTA_EM              0
NOTA_MF              0
NOTA_GO           3716
H_AULA_PRES          0
TAREFAS_ONLINE       0
FALTAS               0
PERFIL               0
dtype: int64

In [17]:
#Substituindo todos os missing values por 0.
df_data_2 = df_data_2.fillna(0)    
print(df_data_2.isnull().sum(axis = 0))

REPROVACOES_DE    0
REPROVACOES_EM    0
REPROVACOES_MF    0
REPROVACOES_GO    0
NOTA_DE           0
NOTA_EM           0
NOTA_MF           0
NOTA_GO           0
H_AULA_PRES       0
TAREFAS_ONLINE    0
FALTAS            0
PERFIL            0
dtype: int64


In [18]:
df_data_2.head(20)

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,0,0,0,0,6.2,5.8,4.6,5.9,2,4,3,EXATAS
1,0,0,0,0,6.0,6.2,5.2,4.5,2,4,3,EXATAS
2,0,0,0,0,7.3,6.7,7.1,7.2,5,0,3,HUMANAS
3,1,3,1,1,0.0,0.0,0.0,0.0,4,4,4,DIFICULDADE
4,1,3,1,1,0.0,0.0,0.0,0.0,5,2,5,DIFICULDADE
5,0,0,0,0,7.3,7.4,7.6,6.5,5,3,5,HUMANAS
6,0,0,0,0,5.8,6.0,7.3,5.1,5,2,6,DIFICULDADE
7,0,0,0,0,4.9,5.0,5.9,4.6,2,2,6,DIFICULDADE
8,0,0,0,0,4.4,4.8,4.7,4.6,3,4,4,DIFICULDADE
9,0,0,0,0,6.4,5.4,5.0,5.5,3,5,3,EXATAS


In [19]:
#Descrição da minha base de dados
df_data_2.describe()

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,0.242850,0.242850,0.297650,0.293350,5.196560,5.080285,4.817630,3.691665,5.102950,3.140300,4.060600
std,0.615055,0.615055,0.672813,0.661829,2.522545,2.523928,2.734775,2.869913,4.118422,1.854909,1.674714
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,5.200000,4.900000,4.700000,0.000000,2.000000,2.000000,3.000000
50%,0.000000,0.000000,0.000000,0.000000,6.200000,5.900000,5.500000,5.000000,4.000000,3.000000,4.000000
75%,0.000000,0.000000,0.000000,0.000000,6.700000,6.700000,6.500000,6.000000,6.000000,5.000000,6.000000
max,3.000000,3.000000,3.000000,3.000000,9.000000,9.400000,11.500000,10.000000,25.000000,7.000000,8.000000


In [20]:
#df_data_1["NOTA_GO"].fillna(df_data_1["NOTA_GO"].median(), inplace=True)
#df_data_1['INGLES'].fillna(df_data_1["INGLES"].replace(np.nan, 0))


Removendo totas as linhas onde a coluna H_AULA_PRES & TAREFAS_ONLINE == 0 & FALTAS == 8
Estou assumindo que uma pessoa que teve resultados como esses não estava matriculado na matéria.

In [21]:
#df_remove = df_data_3.loc[(df_data_3['H_AULA_PRES'] == 0) 
#                         & (df_data_3['TAREFAS_ONLINE'] == 0) 
#                         & (df_data_3['FALTAS'] == 8)]
#ultimo_df = df_data_3.drop(df_remove.index)

In [22]:
#df_data_3 = ultimo_df

In [23]:
print("Nota máx da matéria NOTA_MF",df_data_2["NOTA_MF"].max())
print("Nota máx da matéria NOTA_GO",df_data_2["NOTA_GO"].max())
print("Nota máx da matéria NOTA_DE",df_data_2["NOTA_DE"].max())
print("Nota máx da matéria NOTA_EM",df_data_2["NOTA_EM"].max())

Nota máx da matéria NOTA_MF 11.5
Nota máx da matéria NOTA_GO 10.0
Nota máx da matéria NOTA_DE 9.0
Nota máx da matéria NOTA_EM 9.4


In [24]:
df_data_2.loc[df_data_2["NOTA_MF"] > 10, "NOTA_MF"] = 10

In [25]:
df_data_2['NOTA_MF'].max()

10.0

In [26]:
si = SimpleImputer(
    missing_values=np.nan,  
    strategy='constant', 
    fill_value=0,   
    verbose=0,
    copy=True
)

In [27]:
si

SimpleImputer(fill_value=0, strategy='constant')

In [28]:
# Aplicamos o SimpleImputer ``si`` ao conjunto de dados df_data_2 (resultado da primeira transformação)
si.fit(X=df_data_2)

# Reconstrução de um novo DataFrame Pandas com o conjunto imputado (df_data_3)
df_data_final = pd.DataFrame.from_records(
    data=si.transform(
        X=df_data_2
    ),  # o resultado SimpleImputer.transform(<<pandas dataframe>>) é lista de listas
    columns=df_data_2.columns  # as colunas originais devem ser conservadas nessa transformação
)

In [29]:
df_data_final

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,0,0,0,0,6.2,5.8,4.6,5.9,2,4,3,EXATAS
1,0,0,0,0,6.0,6.2,5.2,4.5,2,4,3,EXATAS
2,0,0,0,0,7.3,6.7,7.1,7.2,5,0,3,HUMANAS
3,1,3,1,1,0.0,0.0,0.0,0.0,4,4,4,DIFICULDADE
4,1,3,1,1,0.0,0.0,0.0,0.0,5,2,5,DIFICULDADE
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,0,0,0,7.0,7.9,5.8,7.0,9,5,6,EXATAS
19996,1,1,1,1,0.0,0.0,0.0,0.0,3,2,5,DIFICULDADE
19997,0,0,0,0,7.9,7.6,8.3,7.2,8,3,1,EXCELENTE
19998,0,0,1,1,6.3,5.1,0.0,0.0,3,2,6,DIFICULDADE


In [30]:
# Definição das colunas que serão features
features = [
    'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_final[features]
y = df_data_final[target]

In [31]:
X.head()

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
0,0,0,0,0,6.2,5.8,4.6,5.9,2,4,3
1,0,0,0,0,6.0,6.2,5.2,4.5,2,4,3
2,0,0,0,0,7.3,6.7,7.1,7.2,5,0,3
3,1,3,1,1,0.0,0.0,0.0,0.0,4,4,4
4,1,3,1,1,0.0,0.0,0.0,0.0,5,2,5


In [32]:
y.head()

,PERFIL
0,EXATAS
1,EXATAS
2,HUMANAS
3,DIFICULDADE
4,DIFICULDADE


In [33]:
# Separação dos dados em um conjunto de treino e um conjunto de teste

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)


In [34]:
modelo = XGBClassifier()
modelo.fit(X_train, y_train) 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


XGBClassifier(objective='multi:softprob')

In [35]:
y_pred = modelo.predict(X_test)


In [36]:
X_test = pd.DataFrame(X_test, columns=['REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS",])
X_test.head()

,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,H_AULA_PRES,TAREFAS_ONLINE,FALTAS
5414,0,0,0,0,5.9,5.4,4.6,4.9,0,1,6
7475,0,0,1,1,7.1,6.8,0.0,0.0,5,6,2
17254,1,1,3,1,0.0,0.0,0.0,0.0,2,2,3
1716,0,0,0,0,6.6,6.7,5.1,0.0,3,2,6
18538,0,0,0,0,5.8,6.1,5.4,5.0,9,2,6


In [37]:
print(y_pred)

['DIFICULDADE' 'DIFICULDADE' 'DIFICULDADE' ... 'EXATAS' 'EXATAS' 'EXATAS']


In [38]:
from sklearn.metrics import accuracy_score

# Acurácia alcançada pela árvore de decisão
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))



Acurácia: 82.0%


In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

 DIFICULDADE       0.90      0.74      0.81      2111
      EXATAS       0.80      0.92      0.86      2455
   EXCELENTE       0.63      0.71      0.67       217
     HUMANAS       0.77      0.95      0.85       937
   MUITO_BOM       0.46      0.05      0.08       280

    accuracy                           0.81      6000
   macro avg       0.71      0.67      0.66      6000
weighted avg       0.81      0.81      0.80      6000

